In [1]:

"""
The main code for the recurrent and convolutional networks assignment.
See README.md for details.
"""
from typing import Tuple, List, Dict
import tensorflow
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import regularizers


def create_toy_rnn(input_shape: tuple, n_outputs: int) \
        -> Tuple[tensorflow.keras.models.Model, Dict]:
    """Creates a recurrent neural network for a toy problem.

    The network will take as input a sequence of number pairs, (x_{t}, y_{t}),
    where t is the time step. It must learn to produce x_{t-3} - y{t} as the
    output of time step t.

    This method does not call Model.fit, but the dictionary it returns alongside
    the model will be passed as extra arguments whenever Model.fit is called.
    This can be used to, for example, set the batch size or use early stopping.

    :param input_shape: The shape of the inputs to the model.
    :param n_outputs: The number of outputs from the model.
    :return: A tuple of (neural network, Model.fit keyword arguments)
    """
    ### YOUR CODE HERE ###

    # Define the input layer with the specified input shape
    input_layer = tf.keras.Input(shape=input_shape)

    # First LSTM layer: 256 units (neurons)
    x = tf.keras.layers.LSTM(256, return_sequences=True, activation='tanh')(input_layer) # 'tanh' activation function
    # Apply dropout to prevent overfitting (drop 30% of the units)
    x = tf.keras.layers.Dropout(0.3)(x)

    # Second LSTM layer: 128 units
    x = tf.keras.layers.LSTM(128, return_sequences=True, activation='tanh')(x)
    # Apply dropout
    x = tf.keras.layers.Dropout(0.3)(x)

    # Third LSTM layer: 64 units
    x = tf.keras.layers.LSTM(64, return_sequences=True, activation='tanh')(x)
    # Apply dropout
    x = tf.keras.layers.Dropout(0.3)(x)

    # Output layer: Dense layer with 'linear' activation for regression output
    output_layer = tf.keras.layers.Dense(
        n_outputs,
        activation='linear',
        kernel_regularizer=regularizers.l2(0.001) # L2 regularization applied to the kernel weights to reduce overfitting
    )(x)

    # Create the model specifying the inputs and outputs
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    # Compile the model: Use RMSprop optimizer with a small learning rate and gradient clipping
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=['mae'])

    # Define training arguments:
    fit_args = {
        'batch_size': 4, # Small batch size for more frequent weight updates
        'epochs': 100, # Increased number of epochs to allow the model to learn sufficiently
        'callbacks': [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15, # EarlyStopping callback to prevent overfitting
                restore_best_weights=True
            )
        ]
    }

    # Return the compiled model and the training arguments
    return model, fit_args

In [2]:
def create_mnist_cnn(input_shape: tuple, n_outputs: int) \
        -> Tuple[tensorflow.keras.models.Model, Dict]:
    """Creates a convolutional neural network for digit classification.

    The network will take as input a 28x28 grayscale image, and produce as
    output one of the digits 0 through 9. The network will be trained and tested
    on a fraction of the MNIST data: http://yann.lecun.com/exdb/mnist/

    This method does not call Model.fit, but the dictionary it returns alongside
    the model will be passed as extra arguments whenever Model.fit is called.
    This can be used to, for example, set the batch size or use early stopping.

    :param input_shape: The shape of the inputs to the model.
    :param n_outputs: The number of outputs from the model.
    :return: A tuple of (neural network, Model.fit keyword arguments)
    """
    ### YOUR CODE HERE ###

    # Input layer for 28x28 grayscale images
    mnist_input = tf.keras.Input(shape=input_shape)

    # First convolutional layer with 32 filters and a 3x3 kernel
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(mnist_input)

    # MaxPooling to reduce spatial dimensions
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Second convolutional layer with 64 filters and a 3x3 kernel
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)

    # MaxPooling to further reduce spatial dimensions
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Flatten the output from 2D to 1D for fully connected layers
    flat_layer = tf.keras.layers.Flatten()(pool2)

    # Dense layer with 128 units for feature learning
    dense_layer = tf.keras.layers.Dense(128, activation='relu')(flat_layer)

    # Output layer with 'softmax' activation for classification into 10 digits
    output_layer = tf.keras.layers.Dense(n_outputs, activation='softmax')(dense_layer)

    # Create the model
    mnist_cnn_model = tf.keras.Model(inputs=mnist_input, outputs=output_layer)

    # Compile the model with categorical crossentropy for classification
    mnist_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Arguments for model training (e.g., batch size, early stopping).
    fit_args = {
        'batch_size': 64,  # Batch size for training
        'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]  # Early stopping
    }

    # Return the model and training arguments
    return mnist_cnn_model, fit_args


In [3]:
def create_youtube_comment_rnn(vocabulary: List[str], n_outputs: int) \
        -> Tuple[tensorflow.keras.models.Model, Dict]:
    """Creates a recurrent neural network for spam classification.

    This network will take as input a YouTube comment, and produce as output
    either 1, for spam, or 0, for ham (non-spam). The network will be trained
    and tested on data from:
    https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection

    Each comment is represented as a series of tokens, with each token
    represented by a number, which is its index in the vocabulary. Note that
    comments may be of variable length, so in the input matrix, comments with
    fewer tokens than the matrix width will be right-padded with zeros.

    This method does not call Model.fit, but the dictionary it returns alongside
    the model will be passed as extra arguments whenever Model.fit is called.
    This can be used to, for example, set the batch size or use early stopping.

    :param vocabulary: The vocabulary defining token indexes.
    :param n_outputs: The number of outputs from the model.
    :return: A tuple of (neural network, Model.fit keyword arguments)
    """
    ### YOUR CODE HERE ###

    # Size of the vocabulary (number of unique tokens)
    vocab_size = len(vocabulary)
    # Dimension for word embeddings (each word will be represented by a 300-dimensional vector)
    embedding_dim = 300
    # Maximum length of a comment (number of tokens).
    max_comment_length = 200

    # Define the input layer
    youtube_input = tf.keras.Input(shape=(max_comment_length,), dtype='int32')

    # Embedding layer
    embedding_layer = tf.keras.layers.Embedding(
        input_dim=vocab_size,          # Size of the vocabulary
        output_dim=embedding_dim       # Dimension of the embedding vectors
    )(youtube_input)

    # First LSTM layer
    lstm_layer = tf.keras.layers.LSTM(
        128, # Number of LSTM cells
        return_sequences=True,
        activation='tanh' # Activation function for the LSTM cells
    )(embedding_layer)

    # Bidirectional LSTM layer (processes the sequence in both forward and backward directions)
    bi_lstm_layer = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            128, # - 128 units for each direction
            return_sequences=False,
            activation='tanh'
        )
    )(lstm_layer)

    # Dense layer with 64 neurons and ReLU activation
    dense_layer = tf.keras.layers.Dense(
        64,
        activation='relu'
    )(bi_lstm_layer)

    # Output layer with sigmoid activation for binary classification
    # Outputs a probability between 0 and 1 indicating spam likelihood
    output_layer = tf.keras.layers.Dense(
        n_outputs,
        activation='sigmoid'
    )(dense_layer)

    # Create the Keras model
    model = tf.keras.Model(inputs=youtube_input, outputs=output_layer)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), # Adam with a low learning rate for fine-tuned weight updates
        loss='binary_crossentropy', # Binary Crossentropy for binary classification tasks
        metrics=['accuracy']
    )

    # Additional arguments for model training
    fit_args = {
        'batch_size': 32, # batch_size
        'epochs': 20, # epochs i.e Number of complete passes through the training dataset
        'callbacks': [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',    # Monitor the validation loss
                patience=5,            # Number of epochs with no improvement after which training will be stopped
                restore_best_weights=True
            )
        ]
    }

    # Return the compiled model and the training arguments
    return model, fit_args


In [4]:
def create_youtube_comment_cnn(vocabulary: List[str], n_outputs: int) \
        -> Tuple[tensorflow.keras.models.Model, Dict]:
    """Creates a convolutional neural network for spam classification.

    This network will take as input a YouTube comment, and produce as output
    either 1, for spam, or 0, for ham (non-spam). The network will be trained
    and tested on data from:
    https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection

    Each comment is represented as a series of tokens, with each token
    represented by a number, which is its index in the vocabulary. Note that
    comments may be of variable length, so in the input matrix, comments with
    fewer tokens than the matrix width will be right-padded with zeros.

    This method does not call Model.fit, but the dictionary it returns alongside
    the model will be passed as extra arguments whenever Model.fit is called.
    This can be used to, for example, set the batch size or use early stopping.

    :param vocabulary: The vocabulary defining token indexes.
    :param n_outputs: The number of outputs from the model.
    :return: A tuple of (neural network, Model.fit keyword arguments)
    """
    ### YOUR CODE HERE ###

    vocab_size = len(vocabulary)  # Number of unique words in vocabulary
    embedding_dim = 128  # Dimension of word embeddings
    max_comment_length = 200  # Maximum length of comments

    # Input layer for tokenized comments
    youtube_input = tf.keras.Input(shape=(max_comment_length,), dtype='int32')

    # Embedding layer to convert tokens into dense vectors
    embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(youtube_input)

    # Convolutional layer with ReLU activation
    conv_layer = tf.keras.layers.Conv1D(128, 5, activation='relu')(embedding_layer)

    # MaxPooling layer to reduce the size of the feature map
    max_pooling_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)

    # Dense layer for classification
    dense_layer = tf.keras.layers.Dense(64, activation='relu')(max_pooling_layer)
    output_layer = tf.keras.layers.Dense(n_outputs, activation='sigmoid')(dense_layer)

    # Create and compile the CNN model
    model = tf.keras.Model(inputs=youtube_input, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Additional arguments for Model.fit
    fit_args = {
        'batch_size': 64, # Batch size
        'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]  # Stop early if no improvement
    }

    return model, fit_args